<a href="https://colab.research.google.com/github/SaiDeepika2004/Alzheimers_disease_detection/blob/main/FINAL_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install imblearn

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout, Input, Concatenate, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
# Load dataset
df = pd.read_csv("alzheimers_disease_data.csv")

# Drop non-relevant columns
df.drop(columns=["PatientID", "DoctorInCharge"], inplace=True)

In [4]:
# Encode categorical variables
le = LabelEncoder()
categorical_columns = ["Gender", "Ethnicity"]
for col in categorical_columns:
    df[col] = le.fit_transform(df[col])

# Split features and target
X = df.drop(columns=["Diagnosis"])
y = df["Diagnosis"]

In [5]:
# Standardize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
# Handle Class Imbalance using SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

In [7]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [8]:
# Feature Selection using Decision Tree
dt = DecisionTreeClassifier(random_state=42, max_depth=10, min_samples_split=5)
dt.fit(X_train, y_train)
dt_top_features = set(np.argsort(dt.feature_importances_)[-20:])

In [9]:
# Feature Selection using Random Forest
rf = RandomForestClassifier(n_estimators=500, random_state=42, max_depth=15, min_samples_split=4, min_samples_leaf=2, n_jobs=-1)
rf.fit(X_train, y_train)
rf_top_features = set(np.argsort(rf.feature_importances_)[-25:])

In [10]:
# Common features from DT and RF
selected_features = list(dt_top_features.intersection(rf_top_features))

In [11]:
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]

# Reshape for CNN input
X_train_selected_cnn = X_train_selected.reshape(X_train_selected.shape[0], X_train_selected.shape[1], 1)
X_test_selected_cnn = X_test_selected.reshape(X_test_selected.shape[0], X_test_selected.shape[1], 1)

In [12]:
# CNN Model
cnn_input = Input(shape=(X_train_selected.shape[1], 1))
cnn_layer = Conv1D(128, 3, activation='relu', kernel_regularizer=l2(0.0005))(cnn_input)
cnn_layer = BatchNormalization()(cnn_layer)
cnn_layer = Conv1D(256, 3, activation='relu', kernel_regularizer=l2(0.0005))(cnn_layer)
cnn_layer = BatchNormalization()(cnn_layer)
cnn_layer = Flatten()(cnn_layer)
cnn_layer = Dense(512, activation='relu', kernel_regularizer=l2(0.0005))(cnn_layer)
cnn_layer = Dropout(0.4)(cnn_layer)

In [13]:
# ANN Model
ann_input = Input(shape=(X_train_selected.shape[1],))
ann_layer = Dense(512, activation='relu', kernel_regularizer=l2(0.0005))(ann_input)
ann_layer = BatchNormalization()(ann_layer)
ann_layer = Dropout(0.4)(ann_layer)
ann_layer = Dense(256, activation='relu', kernel_regularizer=l2(0.0005))(ann_layer)
ann_layer = Dropout(0.4)(ann_layer)

In [14]:
# Merge CNN and ANN
merged_layer = Concatenate()([cnn_layer, ann_layer])
merged_layer = Dense(512, activation='relu', kernel_regularizer=l2(0.0005))(merged_layer)
merged_layer = Dropout(0.4)(merged_layer)
output_layer = Dense(1, activation='sigmoid')(merged_layer)

In [15]:
# Create Model
final_model = Model(inputs=[cnn_input, ann_input], outputs=output_layer)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003)
final_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=30, min_lr=1e-6)

In [17]:
# !pip uninstall tensorflow -y
# !pip install tensorflow

In [18]:
# Train Model
history = final_model.fit([X_train_selected_cnn, X_train_selected], y_train, epochs=150, batch_size=16, validation_data=([X_test_selected_cnn, X_test_selected], y_test), callbacks=[early_stopping, reduce_lr])

Epoch 1/150
139/139 ━━━━━━━━━━━━━━━━━━━━ 19s 107ms/step - accuracy: 0.7309 - loss: 1.6756 - val_accuracy: 0.5719 - val_loss: 1.6202 - learning_rate: 3.0000e-04
Epoch 2/150
139/139 ━━━━━━━━━━━━━━━━━━━━ 15s 105ms/step - accuracy: 0.8497 - loss: 1.3998 - val_accuracy: 0.7500 - val_loss: 1.4887 - learning_rate: 3.0000e-04
Epoch 3/150
139/139 ━━━━━━━━━━━━━━━━━━━━ 14s 101ms/step - accuracy: 0.8739 - loss: 1.3384 - val_accuracy: 0.8147 - val_loss: 1.3840 - learning_rate: 3.0000e-04
Epoch 4/150
139/139 ━━━━━━━━━━━━━━━━━━━━ 14s 102ms/step - accuracy: 0.9021 - loss: 1.2574 - val_accuracy: 0.9173 - val_loss: 1.2079 - learning_rate: 3.0000e-04
Epoch 5/150
139/139 ━━━━━━━━━━━━━━━━━━━━ 14s 102ms/step - accuracy: 0.9194 - loss: 1.1817 - val_accuracy: 0.9029 - val_loss: 1.2139 - learning_rate: 3.0000e-04
Epoch 6/150
139/139 ━━━━━━━━━━━━━━━━━━━━ 14s 103ms/step - accuracy: 0.9401 - loss: 1.1226 - val_accuracy: 0.9173 - val_loss: 1.1648 - learning_rate: 3.0000e-04
Epoch 7/150
139/139 ━━━━━━━━━━━━━━━━━━━━

In [19]:
# Evaluate
y_pred = (final_model.predict([X_test_selected_cnn, X_test_selected]) > 0.5).astype(int)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Final Accuracy
_, accuracy = final_model.evaluate([X_test_selected_cnn, X_test_selected], y_test)
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.93      0.95       278
           1       0.93      0.96      0.95       278

    accuracy                           0.95       556
   macro avg       0.95      0.95      0.95       556
weighted avg       0.95      0.95      0.95       556

Confusion Matrix:
[[258  20]
 [ 10 268]]
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9363 - loss: 0.5630 
Final Test Accuracy: 94.60%
